In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import scipy.stats
import sys
sys.path.insert(1, '/glade/u/home/lettier/analysis/')
import master_utils as myf
xr.set_options(keep_attrs=True)

In [ ]:
myvariables = ['U','V','TREFHT','PS']
mytime = pd.date_range(start="1979-01-01",end="2018-12-31", freq='M')
eddir = '/glade/scratch/wriggles/archive/'
mydir = '/glade/work/lettier/NUDGE/'

In [ ]:
def wrangle_nudge (nudge_name, myvariables):
    ds_a = xr.open_mfdataset(eddir+nudge_name+'/atm/hist/*.h0.*')[myvariables].isel(lev=-1)
    ds_b = xr.open_mfdataset(eddir+nudge_name+'_21C/atm/hist/*.h0.*')[myvariables].isel(lev=-1)
    ds = xr.concat([ds_a,ds_b],dim='time').sel(time=slice('1979-02','2019-01'))
    ds['time'] = mytime
    ds['names'] = nudge_name
    ds = ds.set_coords('names')
    
    return ds

In [ ]:
def wrangle_proc_nudge(nudge_name, myvariables):

    listds = []
    for var in myvariables:
        listds.append(xr.open_dataset('/glade/campaign/univ/uwas0104/sosi_nudge_era/'+nudge_name+'/atm/proc/tseries/month_1/'+nudge_name+'.cam.h0.'+var+'.197901-200512.nc').isel(lev=-1))
    ds_a = xr.merge(listds)

    listds = []
    for var in myvariables:
        listds.append(xr.open_dataset('/glade/campaign/univ/uwas0104/sosi_nudge_era/'+nudge_name+'_21C/atm/proc/tseries/month_1/'+nudge_name+'_21C.cam.h0.'+var+'.200601-201812.nc').isel(lev=-1))
    ds_b = xr.merge(listds)

    ds = xr.concat([ds_a,ds_b],dim='time')
    ds['time'] = mytime
    ds['names'] = nudge_name
    ds = ds.set_coords('names')
    ds = ds[myvariables]

    return ds

In [ ]:
def wrangle_lens (e, myvariables):
    ledir = '/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/'

    ens = str(e)
    if e<10:
        ens = '0'+str(e)
    tmp = []
    print(ens)
    for var in myvariables:
        myfiles = sorted([ledir+var+'/'+f for f in os.listdir(ledir+var) if ('B20TRC5CNBDRD' in f or 'BRCP85C5CNBDRD' in f) and '0'+ens+'.cam.h0.'+var in f ])
        myfiles = [f for f in myfiles if '.192001-199912.nc' not in f and '208101-210012.nc' not in f]
        ds = xr.open_mfdataset(myfiles)[[var]].sel(time=slice('1979-02','2019-01'))
        if 'lev' in ds:
            ds = ds.isel(lev=-1)
        ds['time'] = mytime
        tmp.append(ds)
    ds = xr.merge(tmp)
    ds['names'] = 'LENS'+ens
    ds = ds.set_coords('names')

    return ds

In [ ]:
def get_spatial_stuff (ds):
    name = str(ds.names.values)
    print(name)
    ds['windspeed'] = xr.ufuncs.sqrt(ds.U**2.+ds.V**2.)
    ds.windspeed.attrs['units'] = ds.U.attrs['units']
 
    clim = ds.groupby('time.month').mean(dim='time')
    for var in ['U','V','TREFHT','PS','windspeed']:
        clim = clim.rename({var:var+'_clim'})
    
    slist = []
    for s, seas in enumerate(['JJA','DJF','SON','MAM']):
        mydata = ds.where(ds['time.season'] == seas).groupby('time.year').mean(dim='time')
        slope, intercept, r_value, p_value, std_err = myf.linregress(np.arange(len(mytime)/12),mydata.load(),dim='year')

        for var in ['U','V','TREFHT','PS','windspeed']:
            slope[var].attrs['units'] = clim[var+'_clim'].attrs['units']+'/yr'
            p_value[var] = 100.*p_value[var]
            p_value[var].attrs['units'] = '%'
            slope = slope.rename({var:var+'_trend'})
            p_value = p_value.rename({var:var+'_p_value'})

            
        sds = xr.merge([slope, p_value])
        sds['season'] = seas
        sds = sds.set_coords('season')
        slist.append(sds)
    sds = xr.concat(slist, dim='season')
        
        
    ds = xr.merge([clim, sds])

    ds.attrs['desc'] = 'processed by Lettie Roach, Aug 2021'
    ds.to_netcdf(mydir+'processed/spatial_mean_trend/atm/'+name+'.atm_climtrend.1979-2018.nc')
    
    return ds    
    

All LENS members

In [ ]:
for e in range(1,36,1):
    get_spatial_stuff(wrangle_lens(e,myvariables))

Nudge runs

In [ ]:
for name in ['anom_nudge_era_60_arclo']:
    ds = get_spatial_stuff (wrangle_nudge(name, myvariables))

ERA Interim

In [ ]:
obds = xr.open_dataset('/glade/work/lettier/ERAI/mon/remap_cesmgrid/ei.moda.an.sfc.regn128sc.1979-2018_ptuv_remapcesmagrid.nc')[['t2m','sp']]
obds['time'] = mytime
obds = obds.rename({'t2m':'TREFHT','sp':'PS'})

listds = []
for y in np.arange(1979,2019,1):
    for m in range(1,13,1):
        y, m = str(y), str(m)
        if len(m)==1:
            m = '0'+m
        tmpds = xr.open_dataset('/glade/scratch/lettier/ERA-I-forcing/yrmonmean/ERA-I.6hour.sfc.uv.'+y+'-'+m+'.nc')
        tmpds['time'] = pd.to_datetime(y+'-'+m+'-01')
        tmpds = tmpds.set_coords('time')
        listds.append(tmpds)
uvds = xr.concat(listds, dim='time')
uvds['time'] = mytime
uvds.U.attrs['units'] = 'm/s'
uvds.V.attrs['units'] = 'm/s'
uvds['windspeed'] = xr.ufuncs.sqrt(uvds.U**2.+uvds.V**2.)
uvds.windspeed.attrs['units'] = 'm/s'
uvds['lat'] = obds.lat
uvds['lon'] = obds.lon
obds = xr.merge([obds,uvds])

obds['names'] = 'ERAI'
obds = obds.set_coords('names')
ds = get_spatial_stuff(obds)